In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json here


Saving kaggle1.json to kaggle1.json


{'kaggle1.json': b'{"username": "dummy", "key": "not_a_real_key"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c obss-intern-competition-2025


 94% 931M/995M [00:00<00:00, 963MB/s] 
100% 995M/995M [00:00<00:00, 1.13GB/s]


In [ ]:
!unzip obss-intern-competition-2025.zip -d ./dataset


Streaming output truncated to the last 5000 lines.
  inflating: ./dataset/train/train/5499.jpg  
  inflating: ./dataset/train/train/55.jpg  
  inflating: ./dataset/train/train/550.jpg  
  inflating: ./dataset/train/train/5500.jpg  
  inflating: ./dataset/train/train/5501.jpg  
  inflating: ./dataset/train/train/5502.jpg  
  inflating: ./dataset/train/train/5503.jpg  
  inflating: ./dataset/train/train/5504.jpg  
  inflating: ./dataset/train/train/5505.jpg  
  inflating: ./dataset/train/train/5506.jpg  
  inflating: ./dataset/train/train/5507.jpg  
  inflating: ./dataset/train/train/5508.jpg  
  inflating: ./dataset/train/train/5509.jpg  
  inflating: ./dataset/train/train/551.jpg  
  inflating: ./dataset/train/train/5510.jpg  
  inflating: ./dataset/train/train/5511.jpg  
  inflating: ./dataset/train/train/5512.jpg  
  inflating: ./dataset/train/train/5513.jpg  
  inflating: ./dataset/train/train/5514.jpg  
  inflating: ./dataset/train/train/5515.jpg  
  inflating: ./dataset/train/trai

In [ ]:
import torch

# Load the processed dataset (don't load the whole thing into memory if it's huge!)
data = torch.load("/content/dataset/processed_train.pt")

print(f"Total samples: {len(data)}")

# Just check the first sample's keys and shapes, no heavy operations
sample = data[0]
print("Sample keys:", sample.keys())
print("Pixel values shape:", sample['pixel_values'].shape)
print("Input IDs shape:", sample['input_ids'].shape)
print("Attention mask shape:", sample['attention_mask'].shape)
print("Labels shape:", sample['labels'].shape)


Total samples: 21367
Sample keys: dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'labels'])
Pixel values shape: torch.Size([3, 384, 384])
Input IDs shape: torch.Size([128])
Attention mask shape: torch.Size([128])
Labels shape: torch.Size([128])


In [ ]:
import os
import pandas as pd
from PIL import Image
from transformers import BlipProcessor
import torch

# Paths
csv_path = "/content/dataset/train.csv"
images_dir = "/content/dataset/train/train"
processed_data_path = "/content/dataset/processed_train.pt"

# Load CSV and processor
df = pd.read_csv(csv_path)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")

processed_dataset = []

for idx, row in df.iterrows():
    image_id = row['image_id']
    caption = row['caption']
    image_path = os.path.join(images_dir, f"{image_id}.jpg")

    try:
        # Open image
        image = Image.open(image_path).convert("RGB")

        # Process image and text
        inputs = processor(
            images=image,
            text=caption,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        processed_dataset.append({
            "pixel_values": inputs.pixel_values.squeeze(0),  # [3, H, W]
            "input_ids": inputs.input_ids.squeeze(0),        # [max_length]
            "attention_mask": inputs.attention_mask.squeeze(0),
            "labels": inputs.input_ids.squeeze(0),           # for training target
        })

    except Exception as e:
        print(f"Skipping corrupted image {image_id}: {e}")

# Save processed dataset for easy reload
torch.save(processed_dataset, processed_data_path)
print(f"Saved processed dataset at {processed_data_path}")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Saved processed dataset at /content/dataset/processed_train.pt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import torch
from torch.utils.data import Dataset
from transformers import BlipProcessor, BlipForConditionalGeneration, Trainer, TrainingArguments
from transformers import default_data_collator
from sklearn.model_selection import train_test_split

# Dataset class to wrap your processed data
class PreprocessedBlipDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return {
            "pixel_values": sample['pixel_values'],
            "input_ids": sample['input_ids'],
            "attention_mask": sample['attention_mask'],
            "labels": sample['labels'],
        }

# Load your processed dataset
data = torch.load("/content/dataset/processed_train.pt")

# Split into train and validation sets (90% train, 10% val)
train_size = int(0.9 * len(data))
train_dataset = PreprocessedBlipDataset(data[:train_size])
val_dataset = PreprocessedBlipDataset(data[train_size:])

# Load BLIP processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/output/blip-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.05,
    warmup_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    logging_dir="/content/output/blip-finetuned/logs",
    logging_steps=100,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    remove_unused_columns=False,
    max_grad_norm=1.0,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
)

# Start training
trainer.train()

# Save final model & processor
trainer.save_model("/content/output/blip-finetuned")
processor.save_pretrained("/content/output/blip-finetuned")


<ipython-input-10-1b1f7e8feb9b>:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.358000,0.338881
2,0.243700,0.318798
3,0.152200,0.322077


There were missing keys in the checkpoint model loaded: ['text_decoder.cls.predictions.decoder.bias'].


[]

In [ ]:
import shutil

# Copy the final best model
shutil.copytree(
    "/content/output/blip-finetuned",
    "/content/drive/MyDrive/blip-best-model"
)

print("Best model copied to Google Drive.")


Best model copied to Google Drive.


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from PIL import Image
import os

# Load from Google Drive path where you saved it
model_path = "/content/drive/MyDrive/blip-best-model"

processor = BlipProcessor.from_pretrained(model_path)
model = BlipForConditionalGeneration.from_pretrained(model_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-23): 24 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (projection): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1024,),

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

# === Paths ===
test_csv_path = "/content/dataset/test.csv"
test_images_folder = "/content/dataset/test/test"
checkpoint_path = "/content/drive/MyDrive/blip-best-model"  # ✅ Your best model folder
submission_path = "/content/submission.csv"

# === Load model and processor ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = BlipProcessor.from_pretrained(checkpoint_path)
model = BlipForConditionalGeneration.from_pretrained(checkpoint_path).to(device)
model.eval()

# === Load test dataset ===
test_df = pd.read_csv(test_csv_path)

# === Generate captions ===
captions = []
with torch.no_grad():
    for idx, row in test_df.iterrows():
        image_id = row['image_id']
        image_path = os.path.join(test_images_folder, f"{image_id}.jpg")

        image = Image.open(image_path).convert("RGB")
        inputs = processor(image, return_tensors="pt").to(device)

        output_ids = model.generate(**inputs, max_length=30)
        caption = processor.decode(output_ids[0], skip_special_tokens=True)
        captions.append(caption)

        if idx % 100 == 0:
            print(f"Processed {idx}/{len(test_df)} images")

# === Save submission ===
submission_df = pd.DataFrame({
    "image_id": test_df["image_id"],
    "caption": captions
})
submission_df.to_csv(submission_path, index=False)

print(f"✅ Submission saved to: {submission_path}")


Processed 0/3771 images
Processed 100/3771 images
Processed 200/3771 images
Processed 300/3771 images
Processed 400/3771 images
Processed 500/3771 images
Processed 600/3771 images
Processed 700/3771 images
Processed 800/3771 images
Processed 900/3771 images
Processed 1000/3771 images
Processed 1100/3771 images
Processed 1200/3771 images
Processed 1300/3771 images
Processed 1400/3771 images
Processed 1500/3771 images
Processed 1600/3771 images
Processed 1700/3771 images
Processed 1800/3771 images
Processed 1900/3771 images
Processed 2000/3771 images
Processed 2100/3771 images
Processed 2200/3771 images
Processed 2300/3771 images
Processed 2400/3771 images
Processed 2500/3771 images
Processed 2600/3771 images
Processed 2700/3771 images
Processed 2800/3771 images
Processed 2900/3771 images
Processed 3000/3771 images
Processed 3100/3771 images
Processed 3200/3771 images
Processed 3300/3771 images
Processed 3400/3771 images
Processed 3500/3771 images
Processed 3600/3771 images
Processed 370

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

# === Paths ===
test_csv_path = "/content/dataset/test.csv"
test_images_folder = "/content/dataset/test/test"
checkpoint_path = "/content/drive/MyDrive/blip-best-model"  # ✅ Your best model folder
submission_path = "/content/submission1.csv"

# === Load model and processor ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = BlipProcessor.from_pretrained(checkpoint_path)
model = BlipForConditionalGeneration.from_pretrained(checkpoint_path).to(device)
model.eval()

# === Load test dataset ===
test_df = pd.read_csv(test_csv_path)

# === Generate captions ===
captions = []
with torch.no_grad():
    for idx, row in test_df.iterrows():
        image_id = row['image_id']
        image_path = os.path.join(test_images_folder, f"{image_id}.jpg")

        image = Image.open(image_path).convert("RGB")
        inputs = processor(image, return_tensors="pt").to(device)

        output_ids = model.generate(
            **inputs,
            max_length=30,
            num_beams=5,         # Try 3 to 5
            early_stopping=True,
            no_repeat_ngram_size=2
        )

        caption = processor.decode(output_ids[0], skip_special_tokens=True)
        captions.append(caption)

        if idx % 100 == 0:
            print(f"Processed {idx}/{len(test_df)} images")

# === Save submission ===
submission_df = pd.DataFrame({
    "image_id": test_df["image_id"],
    "caption": captions
})
submission_df.to_csv(submission_path, index=False)

print(f"✅ Submission saved to: {submission_path}")


Processed 0/3771 images
Processed 100/3771 images
Processed 200/3771 images
Processed 300/3771 images
Processed 400/3771 images
Processed 500/3771 images
Processed 600/3771 images
Processed 700/3771 images
Processed 800/3771 images
Processed 900/3771 images
Processed 1000/3771 images
Processed 1100/3771 images
Processed 1200/3771 images
Processed 1300/3771 images
Processed 1400/3771 images
Processed 1500/3771 images
Processed 1600/3771 images
Processed 1700/3771 images
Processed 1800/3771 images
Processed 1900/3771 images
Processed 2000/3771 images
Processed 2100/3771 images
Processed 2200/3771 images
Processed 2300/3771 images
Processed 2400/3771 images
Processed 2500/3771 images
Processed 2600/3771 images
Processed 2700/3771 images
Processed 2800/3771 images
Processed 2900/3771 images
Processed 3000/3771 images
Processed 3100/3771 images
Processed 3200/3771 images
Processed 3300/3771 images
Processed 3400/3771 images
Processed 3500/3771 images
Processed 3600/3771 images
Processed 370